In [1]:
import os
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, Column, String, create_engine, MetaData
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy
import logging

app = Flask(__name__)


In [2]:
#app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/pet_adoptions.sqlite"
#db = SQLAlchemy(app)


In [3]:
#Base = automap_base()


In [4]:
#Base.prepare(db.engine, reflect=True)


In [5]:
breed_file= "static/data/breed_labels.csv"
breed_df= pd.read_csv(breed_file)
breed_df.head()

,BreedID,Type,BreedName
0,1,1,Affenpinscher
1,2,1,Afghan Hound
2,3,1,Airedale Terrier
3,4,1,Akbash
4,5,1,Akita


In [6]:

breed_df.describe()


,BreedID,Type
count,307.000000,307.000000
mean,154.000000,1.214984
std,88.767487,0.411482
min,1.000000,1.000000
25%,77.500000,1.000000
50%,154.000000,1.000000
75%,230.500000,1.000000
max,307.000000,2.000000


In [7]:
logging.basicConfig(level=logging.INFO)
logging.getLogger("sqlalchemy.engine").setLevel(logging.INFO)
engine = create_engine("sqlite:///pet_adoptions.sqlite", echo=False)


In [8]:
engine.table_names()

INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
INFO:sqlalchemy.engine.base.Engine:()


['breed_labels', 'color_labels', 'state_labels', 'train']

In [9]:
color_file= "static/data/color_labels.csv"
color_df= pd.read_csv(color_file)
color_df.head()

,ColorID,ColorName
0,1,Black
1,2,Brown
2,3,Golden
3,4,Yellow
4,5,Cream


In [10]:
color_df.describe()

,ColorID
count,7.000000
mean,4.000000
std,2.160247
min,1.000000
25%,2.500000
50%,4.000000
75%,5.500000
max,7.000000


In [11]:
breed_df.to_sql(name='breed_labels', con=engine, if_exists='append', index=True)
color_df.to_sql(name='color_labels', con=engine, if_exists='append', index=True)



INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("breed_labels")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:INSERT INTO breed_labels ("index", "BreedID", "Type", "BreedName") VALUES (?, ?, ?, ?)
INFO:sqlalchemy.engine.base.Engine:((0, 1, 1, 'Affenpinscher'), (1, 2, 1, 'Afghan Hound'), (2, 3, 1, 'Airedale Terrier'), (3, 4, 1, 'Akbash'), (4, 5, 1, 'Akita'), (5, 6, 1, 'Alaskan Malamute'), (6, 7, 1, 'American Bulldog'), (7, 8, 1, 'American Eskimo Dog')  ... displaying 10 of 307 total bound parameter sets ...  (305, 305, 2, 'Turkish Van'), (306, 306, 2, 'Tuxedo'))
INFO:sqlalchemy.engine.base.Engine:COMMIT
INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("color_labels")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:INSERT INTO color_labels ("index", "ColorID", "ColorName") VALUES (?, ?, ?)
INFO:sqlalchemy.engine.base.Engine:(

In [12]:
engine.table_names()

INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
INFO:sqlalchemy.engine.base.Engine:()


['breed_labels', 'color_labels', 'state_labels', 'train']

In [13]:
train_file= "static/data/train.csv"
train_df= pd.read_csv(train_file)
train_df.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3
3,1,Miko,4,307,0,2,1,2,0,2,...,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2
4,1,Hunter,1,307,0,1,1,0,0,2,...,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2


In [14]:
train_df.describe()

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
count,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000,14993.000000
mean,1.457614,10.452078,265.272594,74.009738,1.776162,2.234176,3.222837,1.882012,1.862002,1.467485,1.731208,1.558727,1.914227,1.036617,1.576069,21.259988,41346.028347,0.056760,3.889215,2.516441
std,0.498217,18.155790,60.056818,123.011575,0.681592,1.745225,2.742562,2.984086,0.547959,0.599070,0.667649,0.695817,0.566172,0.199535,1.472477,78.414548,32.444153,0.346185,3.487810,1.177265
min,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,41324.000000,0.000000,0.000000,0.000000
25%,1.000000,2.000000,265.000000,0.000000,1.000000,1.000000,0.000000,0.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,2.000000,2.000000
50%,1.000000,3.000000,266.000000,0.000000,2.000000,2.000000,2.000000,0.000000,2.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,0.000000,41326.000000,0.000000,3.000000,2.000000
75%,2.000000,12.000000,307.000000,179.000000,2.000000,3.000000,6.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.000000,41401.000000,0.000000,5.000000,4.000000
max,2.000000,255.000000,307.000000,307.000000,3.000000,7.000000,7.000000,7.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,20.000000,3000.000000,41415.000000,8.000000,30.000000,4.000000


In [15]:
train_df.to_sql(name='train', con=engine, if_exists='append', index=True)

INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("train")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:INSERT INTO train ("index", "Type", "Name", "Age", "Breed1", "Breed2", "Gender", "Color1", "Color2", "Color3", "MaturitySize", "FurLength", "Vaccinated", "Dewormed", "Sterilized", "Health", "Quantity", "Fee", "State", "RescuerID", "VideoAmt", "Description", "PetID", "PhotoAmt", "AdoptionSpeed") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
INFO:sqlalchemy.engine.base.Engine:((0, 2, 'Nibble', 3, 299, 0, 1, 1, 7, 0, 1, 1, 2, 2, 2, 1, 1, 100, 41326, '8480853f516546f6cf33aa88cd76c379', 0, "Nibble is a 3+ month old ball of cuteness. He is energetic and playful. I rescued a couple of cats a few months ago but could not get them neutered i ... (61 characters truncated) ...  little kitty. I do not have enough space and funds to care for more cats in my household. Looking fo

In [16]:
postcode_file= "static/data/geocoded_state_labels.csv"
postcode_df= pd.read_csv(postcode_file)
postcode_df.head()

,StateID,StateName,LatLong
0,41336,Johor,"1.934400, 103.358727"
1,41325,Kedah,"6.155672, 100.569649"
2,41367,Kelantan,"6.125397, 102.238068"
3,41401,Kuala Lumpur,"3.1412, 101.68653"
4,41415,Labuan,"5.285153, 115.247787"


In [17]:
postcode_df.describe()

,StateID
count,15.000000
mean,41349.733333
std,29.001149
min,41324.000000
25%,41328.500000
50%,41336.000000
75%,41364.000000
max,41415.000000


In [18]:
postcode_df.to_sql(name='state_labels', con=engine, if_exists='append', index=True)

INFO:sqlalchemy.engine.base.Engine:PRAGMA table_info("state_labels")
INFO:sqlalchemy.engine.base.Engine:()
INFO:sqlalchemy.engine.base.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.base.Engine:INSERT INTO state_labels ("index", "StateID", "StateName", "LatLong") VALUES (?, ?, ?, ?)
INFO:sqlalchemy.engine.base.Engine:((0, 41336, 'Johor', '1.934400, 103.358727'), (1, 41325, 'Kedah', '6.155672, 100.569649'), (2, 41367, 'Kelantan', '6.125397, 102.238068'), (3, 41401, 'Kuala Lumpur', '3.1412, 101.68653'), (4, 41415, 'Labuan', '5.285153, 115.247787'), (5, 41324, 'Melaka', '2.200844, 102.240143'), (6, 41332, 'Negeri Sembilan', '2.731813, 102.252502'), (7, 41335, 'Pahang', '3.974341, 102.438057')  ... displaying 10 of 15 total bound parameter sets ...  (13, 41326, 'Selangor', '3.509247, 101.524803'), (14, 41361, 'Terengganu', '5.207053, 103.205299'))
INFO:sqlalchemy.engine.base.Engine:COMMIT


In [19]:
color_id1=engine.execute("SELECT State from train where Color1 = 1")

INFO:sqlalchemy.engine.base.Engine:SELECT State from train where Color1 = 1
INFO:sqlalchemy.engine.base.Engine:()


In [20]:

color_id7=engine.execute("SELECT State from train where Color1 = 7")

INFO:sqlalchemy.engine.base.Engine:SELECT State from train where Color1 = 7
INFO:sqlalchemy.engine.base.Engine:()


In [21]:

color_by_postcode= engine.execute("Select State, Color1, COUNT(COLOR1) from train Group By State, Color1")

INFO:sqlalchemy.engine.base.Engine:Select State, Color1, COUNT(COLOR1) from train Group By State, Color1
INFO:sqlalchemy.engine.base.Engine:()


In [22]:
print(color_by_postcode)
for value in color_by_postcode:
    print(value)

(41324, 1, 318)
(41324, 2, 270)
(41324, 3, 54)
(41324, 4, 30)
(41324, 5, 48)
(41324, 6, 30)
(41324, 7, 72)
(41325, 1, 378)
(41325, 2, 132)
(41325, 3, 72)
(41325, 4, 6)
(41325, 5, 42)
(41325, 6, 12)
(41325, 7, 18)
(41326, 1, 26106)
(41326, 2, 13446)
(41326, 3, 3300)
(41326, 4, 1764)
(41326, 5, 3150)
(41326, 6, 2280)
(41326, 7, 2238)
(41327, 1, 2490)
(41327, 2, 1386)
(41327, 3, 246)
(41327, 4, 192)
(41327, 5, 288)
(41327, 6, 180)
(41327, 7, 276)
(41330, 1, 1254)
(41330, 2, 708)
(41330, 3, 102)
(41330, 4, 78)
(41330, 5, 222)
(41330, 6, 90)
(41330, 7, 66)
(41332, 1, 768)
(41332, 2, 378)
(41332, 3, 60)
(41332, 4, 72)
(41332, 5, 114)
(41332, 6, 60)
(41332, 7, 66)
(41335, 1, 282)
(41335, 2, 102)
(41335, 3, 18)
(41335, 4, 18)
(41335, 5, 18)
(41335, 6, 18)
(41335, 7, 54)
(41336, 1, 1488)
(41336, 2, 720)
(41336, 3, 162)
(41336, 4, 174)
(41336, 5, 162)
(41336, 6, 156)
(41336, 7, 180)
(41342, 1, 66)
(41342, 2, 6)
(41342, 4, 6)
(41345, 1, 54)
(41345, 2, 36)
(41345, 4, 30)
(41345, 6, 12)
(41361, 1, 

In [23]:
breed_by_postcode= engine.execute("Select State, Breed1, COUNT(Breed1) from train GROUP BY State, Breed1 ")


INFO:sqlalchemy.engine.base.Engine:Select State, Breed1, COUNT(Breed1) from train GROUP BY State, Breed1 
INFO:sqlalchemy.engine.base.Engine:()


In [24]:
for value in breed_by_postcode:
    print(value)

(41324, 98, 6)
(41324, 109, 12)
(41324, 141, 6)
(41324, 152, 6)
(41324, 179, 30)
(41324, 190, 6)
(41324, 195, 6)
(41324, 205, 12)
(41324, 218, 6)
(41324, 265, 114)
(41324, 266, 126)
(41324, 285, 36)
(41324, 292, 12)
(41324, 296, 6)
(41324, 299, 6)
(41324, 307, 432)
(41325, 39, 6)
(41325, 103, 18)
(41325, 109, 6)
(41325, 141, 78)
(41325, 152, 6)
(41325, 179, 6)
(41325, 195, 6)
(41325, 205, 6)
(41325, 218, 6)
(41325, 251, 6)
(41325, 254, 6)
(41325, 264, 12)
(41325, 265, 60)
(41325, 266, 174)
(41325, 268, 6)
(41325, 285, 30)
(41325, 289, 6)
(41325, 292, 12)
(41325, 297, 6)
(41325, 307, 204)
(41326, 0, 18)
(41326, 1, 6)
(41326, 3, 6)
(41326, 5, 6)
(41326, 10, 6)
(41326, 15, 6)
(41326, 17, 24)
(41326, 18, 36)
(41326, 19, 24)
(41326, 20, 306)
(41326, 24, 6)
(41326, 25, 6)
(41326, 26, 126)
(41326, 31, 48)
(41326, 32, 12)
(41326, 39, 78)
(41326, 42, 18)
(41326, 44, 42)
(41326, 49, 60)
(41326, 50, 24)
(41326, 56, 6)
(41326, 60, 114)
(41326, 61, 6)
(41326, 64, 6)
(41326, 65, 6)
(41326, 69, 144)


In [25]:
adoptionspeed_by_postcode= engine.execute("Select State, AdoptionSpeed, COUNT(AdoptionSpeed) from train GROUP BY State, AdoptionSpeed ")


INFO:sqlalchemy.engine.base.Engine:Select State, AdoptionSpeed, COUNT(AdoptionSpeed) from train GROUP BY State, AdoptionSpeed 
INFO:sqlalchemy.engine.base.Engine:()


In [26]:
for value in adoptionspeed_by_postcode:
    print(value)

(41324, 0, 24)
(41324, 1, 108)
(41324, 2, 138)
(41324, 3, 72)
(41324, 4, 480)
(41325, 0, 18)
(41325, 1, 84)
(41325, 2, 204)
(41325, 3, 138)
(41325, 4, 216)
(41326, 0, 1476)
(41326, 1, 11262)
(41326, 2, 14610)
(41326, 3, 12024)
(41326, 4, 12912)
(41327, 0, 48)
(41327, 1, 732)
(41327, 2, 1296)
(41327, 3, 1182)
(41327, 4, 1800)
(41330, 0, 18)
(41330, 1, 288)
(41330, 2, 666)
(41330, 3, 702)
(41330, 4, 846)
(41332, 0, 24)
(41332, 1, 216)
(41332, 2, 378)
(41332, 3, 252)
(41332, 4, 648)
(41335, 0, 18)
(41335, 1, 174)
(41335, 2, 84)
(41335, 3, 96)
(41335, 4, 138)
(41336, 0, 138)
(41336, 1, 678)
(41336, 2, 816)
(41336, 3, 618)
(41336, 4, 792)
(41342, 0, 6)
(41342, 1, 6)
(41342, 3, 12)
(41342, 4, 54)
(41345, 0, 6)
(41345, 1, 36)
(41345, 2, 18)
(41345, 3, 24)
(41345, 4, 48)
(41361, 1, 54)
(41361, 2, 12)
(41361, 3, 36)
(41361, 4, 54)
(41367, 0, 12)
(41367, 1, 18)
(41367, 2, 18)
(41367, 3, 6)
(41367, 4, 36)
(41401, 0, 672)
(41401, 1, 4884)
(41401, 2, 5976)
(41401, 3, 4386)
(41401, 4, 7152)
(41415, 

In [27]:
engine.table_names()

INFO:sqlalchemy.engine.base.Engine:SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
INFO:sqlalchemy.engine.base.Engine:()


['breed_labels', 'color_labels', 'state_labels', 'train']

In [28]:
Age_by_postcode= engine.execute("Select State, Age, COUNT(Age) from train GROUP BY State, Age ")


INFO:sqlalchemy.engine.base.Engine:Select State, Age, COUNT(Age) from train GROUP BY State, Age 
INFO:sqlalchemy.engine.base.Engine:()


In [29]:
for value in Age_by_postcode:
    print(value)

(41324, 0, 6)
(41324, 1, 78)
(41324, 2, 174)
(41324, 3, 126)
(41324, 4, 42)
(41324, 5, 36)
(41324, 6, 12)
(41324, 7, 18)
(41324, 8, 18)
(41324, 9, 6)
(41324, 10, 6)
(41324, 12, 78)
(41324, 14, 6)
(41324, 15, 12)
(41324, 18, 18)
(41324, 19, 6)
(41324, 23, 12)
(41324, 24, 78)
(41324, 27, 6)
(41324, 36, 18)
(41324, 42, 6)
(41324, 48, 12)
(41324, 49, 6)
(41324, 54, 6)
(41324, 57, 6)
(41324, 60, 18)
(41324, 108, 6)
(41324, 120, 6)
(41325, 0, 18)
(41325, 1, 120)
(41325, 2, 90)
(41325, 3, 168)
(41325, 4, 48)
(41325, 5, 12)
(41325, 6, 30)
(41325, 9, 12)
(41325, 10, 6)
(41325, 11, 12)
(41325, 12, 18)
(41325, 16, 6)
(41325, 18, 18)
(41325, 20, 12)
(41325, 24, 36)
(41325, 33, 6)
(41325, 36, 6)
(41325, 48, 18)
(41325, 60, 18)
(41325, 87, 6)
(41326, 0, 636)
(41326, 1, 8268)
(41326, 2, 12720)
(41326, 3, 6864)
(41326, 4, 3990)
(41326, 5, 2010)
(41326, 6, 1866)
(41326, 7, 990)
(41326, 8, 1014)
(41326, 9, 600)
(41326, 10, 570)
(41326, 11, 354)
(41326, 12, 3072)
(41326, 13, 126)
(41326, 14, 222)
(41326,